# MNIST Classifier

<center>
    <img alt="MNIST Handwritten Digits Image" src="https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png">
</center>

In this example, we will predict handwritten digits in the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) using a [multi-layer perceptron (MLP)](https://en.wikipedia.org/wiki/Multilayer_perceptron). This is similar to the tutorial notebook, but with an added comparison with a standard model.

Of course, other neural network architectures such as [convolutional neural networks (CNNs)](https://en.wikipedia.org/wiki/Convolutional_neural_network) are better suited for this task, but for this example we will stick with MLPs.

## Setup

First, let's prepare the imports.

In [1]:
import keras
import numpy as np

2024-06-18 14:55:07.504998: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 14:55:07.505336: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 14:55:07.507540: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 14:55:07.533623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 14:55:08.091338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Define constants relating to the data.

In [2]:
NUM_CLASSES = 10        # 10 distinct classes, 0 to 9
INPUT_SHAPE = (28, 28)  # 28 x 28 greyscale images

Load the data from the `mnist` dataset.

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

Perform some preprocessing.

In [4]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

## Defining the Model

As mentioned, we will be using a MLP for the model. However, instead of using `keras`'s default `Dense` layer, we will use `keras_mml`'s `DenseMML` layer (which stands for Dense Matrix-Multiplication-less). `DenseMML` is designed to be a direct replacement for `Dense` layers in fully-connected layers, so we don't have to change the architecture of the model much.

In [5]:
import keras_mml

Define the `Sequential` model.

In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),  # The last layer needs to be `Dense` for the output to work
    ],
    name="Classifier-MML"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
model.summary()

Model: "Classifier-MML"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml (DenseMML)            │ (None, 256)            │       200,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml_1 (DenseMML)          │ (None, 256)            │        65,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml_2 (DenseMML)          │ (None, 256)            │        65,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 334,346 (1.28 MB)

 Trainable params: 334,346 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We can now train the model.

In [8]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4628 - loss: 2.1854 - val_accuracy: 0.7832 - val_loss: 1.3653
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7572 - loss: 1.1981 - val_accuracy: 0.8325 - val_loss: 0.7179
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8057 - loss: 0.7523 - val_accuracy: 0.8568 - val_loss: 0.5401
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8280 - loss: 0.6103 - val_accuracy: 0.8695 - val_loss: 0.4660
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8433 - loss: 0.5490 - val_accuracy: 0.8775 - val_loss: 0.4268
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8535 - loss: 0.5072 - val_accuracy: 0.8840 - val_loss: 0.4018
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8600 - loss: 0.4810 - val_accuracy: 0.8875 - val_loss: 0.3847
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8651 - loss: 0.4631 - val_accuracy: 0.

Once the model is trained, let's evaluate it.

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.36403656005859375
Test accuracy: 0.8956000208854675


Not bad!

## A Comparison - An MLP Using Normal `Dense` Layers

Let's compare our model's performance to a model that uses the regular `Dense` layers.

In [10]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),
    ],
    name="Classifier-Normal"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
model.summary()

Model: "Classifier-Normal"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,114 (1.28 MB)

 Trainable params: 335,114 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We'll train the model using the same `batch_size`, `epochs`, and `validation_split`.

In [12]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8480 - loss: 0.5010 - val_accuracy: 0.9200 - val_loss: 0.2785
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9042 - loss: 0.3342 - val_accuracy: 0.9278 - val_loss: 0.2557
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9096 - loss: 0.3191 - val_accuracy: 0.9303 - val_loss: 0.2498
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9150 - loss: 0.3050 - val_accuracy: 0.9267 - val_loss: 0.2693
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9143 - loss: 0.3051 - val_accuracy: 0.9240 - val_loss: 0.2597
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9148 - loss: 0.2968 - val_accuracy: 0.9245 - val_loss: 0.2818
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9160 - loss: 0.2980 - val_accuracy: 0.9332 - val_loss: 0.2523
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9176 - loss: 0.2934 - val_accuracy: 0.

Again, we evaluate the model.

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.2964845895767212
Test accuracy: 0.9180999994277954


Notice that the accuracy of the normal model is quite close to the accuracy of the MML model. With a slight decrease in accuracy, the model itself does not use matrix multiplications at all, and also reduces memory usage.